In [2]:
from PIL import Image
import pytesseract as tess


In [3]:
im=Image.open("test_image\id.jpg")

In [4]:
print(im)

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x771 at 0x13B128266D0>


In [8]:
image_data=tess.image_to_string(im)
print(image_data)

Independent
We | University,
“Bangladesh

Sourav Kumar Saha
Student
1D: 1930188

el
AMNUUUN) A

